In [ ]:
!pip install -U spacy
!pip install spacy_transformers

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.5.1'

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
!nvidia-smi

Thu Mar 23 09:56:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    27W /  70W |    363MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cv_data = json.load(open('/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Dataset/train_data.json','r'))

In [ ]:
len(cv_data)

200

In [ ]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/base_config.cfg /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/config.cfg

2023-03-23 09:56:28.118219: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 09:56:28.118329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 09:56:28.118349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/config.cfg
You can now add your data and train your pip

In [ ]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start,end))
      try:
        span = doc.char_span(start,end,label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start,end]) + "     " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

(140, 60)

In [ ]:
filee = open('/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Model/train_file.txt','w')

db = get_spacy_doc(filee,train)
db.to_disk('/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Model/train_data.spacy')

db = get_spacy_doc(filee,test)
db.to_disk('/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Model/test_data.spacy')

  1%|          | 1/140 [00:00<00:05, 24.94it/s]


UnicodeEncodeError: ignored

In [ ]:
!python -m spacy train /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/config.cfg --output /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/output --paths.train /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Model/train_data.spacy --paths.dev /content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Model/test_data.spacy --gpu-id 0


2023-03-23 10:05:48.261453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 10:05:48.261560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-23 10:05:48.261579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory:
/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/output
ℹ Using GPU: 0

=========================== Initializing pipeline ====

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/Rafay-Punch-Backup/NER-Resume/Config/output/model-best')

In [ ]:
import sys, fitz

fname = '/content/danish resume-3-2.pdf'
doc = fitz.open(fname)

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())
  

In [ ]:
text

' DANISH\nALI\nI love Being a UI/UX designer as computer science student, it has always been my \ndream to work for a reputable digital services company. To fulfl that dream, I \nstarted learning UI/UX Designing and Graphic Designs from multiple sources \nonline and now I am applying to get a job opportunity so I can polish my  skills \nand excel rapidly in my professional career.\nBachelor of Computer Science\nBahria University\nPre-Engneering \nCivil  Line College Lahore \n2020 - 2021\n2018 - 2022\n2016 - 2018\nKotawa Pakistan Pvt Ltd.\nUI/UX Graphic Designer Intern\nWork with Advance Designing team and get knowledge of \ndiferent tools and technique\nCreate user interface designs and illustration wor diferent \nweb designs \nEXPERIENCE\nEDUCATION\nNursing App (fnal year project) \nLanguages: Dart (Flutter SDK) \nTools: Android Studio, Adobe XD, Firebase \nUI/UX Designs\n       Control system using hand-gestures \n       Banking Management System \n       Fire Fighting Robot \nprojec

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "->>>>>>>>>",ent.label_)

In [ ]:
doc.ents

()